# Data Preparation

1. Import necessary libraries

In [143]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ast

2. Read in the csv-file

In [144]:
charging_data = pd.read_csv("charging_sessions.csv")

## Data Dimensionality

Evaluate the dataset concerning dimensionality, data types and overall structure at first glance. 

In [145]:
charging_data.info(show_counts=True)
charging_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66450 entries, 0 to 66449
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        66450 non-null  int64  
 1   id                66450 non-null  object 
 2   connectionTime    66450 non-null  object 
 3   disconnectTime    66450 non-null  object 
 4   doneChargingTime  62362 non-null  object 
 5   kWhDelivered      66450 non-null  float64
 6   sessionID         66450 non-null  object 
 7   siteID            66450 non-null  int64  
 8   spaceID           66450 non-null  object 
 9   stationID         66450 non-null  object 
 10  timezone          66450 non-null  object 
 11  userID            49187 non-null  float64
 12  userInputs        49187 non-null  object 
dtypes: float64(2), int64(2), object(9)
memory usage: 6.6+ MB


,Unnamed: 0,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs
0,0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1_1_179_810_2020-01-02 13:08:53.870034,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil..."
1,1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1_1_193_825_2020-01-02 13:36:49.599853,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil..."
2,2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1_1_193_829_2020-01-02 13:56:35.214993,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
3,3,5e23b149f9af8b5fe4b973d2,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1_1_193_820_2020-01-02 13:59:58.309319,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
4,4,5e23b149f9af8b5fe4b973d3,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1_1_193_819_2020-01-02 14:00:00.779967,1,AG-1F06,1-1-193-819,America/Los_Angeles,334.0,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'mil..."


## Handle User Inputs

There are three possible scenarios for userInputs:
1. No user inputs
2. Exactly one user input
3. Several user inputs

Possible solutions for handling the userInputs: **TBD**
- Leave the userInput attributes in the list column
- Create one column per userInputs attribute --> Each column contains a list of the input attribute values
- Create one column per userInputs attribute --> Each column contains only the last value for the input attribute

In [146]:
# Example of no userInputs 
print("No user inputs: \n", charging_data[charging_data["userInputs"].isnull()]["userInputs"].iloc[0])

# Example of one userInputs
print("Single user input: \n", charging_data["userInputs"].iloc[0])

# Example of multiple userInputs 
print("Multiple user inputs: \n", charging_data[charging_data["userID"] == 1117]["userInputs"].iloc[0])

No user inputs: 
 nan
Single user input: 
 [{'WhPerMile': 250, 'kWhRequested': 25.0, 'milesRequested': 100, 'minutesAvailable': 463, 'modifiedAt': 'Thu, 02 Jan 2020 13:09:39 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 20:51:54 GMT', 'userID': 194}]
Multiple user inputs: 
 [{'WhPerMile': 400, 'kWhRequested': 8.0, 'milesRequested': 20, 'minutesAvailable': 65, 'modifiedAt': 'Thu, 02 Jan 2020 14:00:03 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 15:04:58 GMT', 'userID': 1117}, {'WhPerMile': 400, 'kWhRequested': 8.0, 'milesRequested': 20, 'minutesAvailable': 65, 'modifiedAt': 'Thu, 02 Jan 2020 14:00:19 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 15:04:58 GMT', 'userID': 1117}]


## Duplicate Values

Check if "id" values are unique

In [147]:
duplicate_ids = charging_data["id"].duplicated(keep="first")
duplicate_ids = len(duplicate_ids[duplicate_ids == True])
duplicate_ids

1413

Hence, there are 1413 duplicate IDs. Further inspection of the duplicates is required before determining how to handle them.

In [148]:
# Collect all duplicated IDs
duplicated_ids = charging_data[charging_data["id"].duplicated()]
duplicated_ids = duplicated_ids["id"].unique()

# Define the subset of variables to inspect for duplicates. Exclude the unnamed index column.
subset_cols = charging_data.columns.difference(["Unnamed: 0"])

duplicates = charging_data[charging_data.duplicated(subset=subset_cols)]
len(duplicates)

1413

This shows that the entries with duplicate IDs also contain duplicate values for all other columns. Hence, the duplicates can be deleted without loss of information. 

In [149]:
charging_data = charging_data.drop_duplicates(subset=subset_cols, keep="first")
charging_data.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 65037 entries, 0 to 65036
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        65037 non-null  int64  
 1   id                65037 non-null  object 
 2   connectionTime    65037 non-null  object 
 3   disconnectTime    65037 non-null  object 
 4   doneChargingTime  60950 non-null  object 
 5   kWhDelivered      65037 non-null  float64
 6   sessionID         65037 non-null  object 
 7   siteID            65037 non-null  int64  
 8   spaceID           65037 non-null  object 
 9   stationID         65037 non-null  object 
 10  timezone          65037 non-null  object 
 11  userID            47822 non-null  float64
 12  userInputs        47822 non-null  object 
dtypes: float64(2), int64(2), object(9)
memory usage: 6.9+ MB


## Missing Values

Check for any missing values within the dataset

In [150]:
missing_values = charging_data.isnull().sum()
missing_values

Unnamed: 0              0
id                      0
connectionTime          0
disconnectTime          0
doneChargingTime     4087
kWhDelivered            0
sessionID               0
siteID                  0
spaceID                 0
stationID               0
timezone                0
userID              17215
userInputs          17215
dtype: int64

Quick analysis of the missing values:
- **"userID"** and **"userInputs"**: Contain missing values as expected, since users are not required to register
- **"doneChargingTime"**: Missing values are expected to be data irregularities and have to be dealt with in order to ensure a high data quality

Subsets of the data are created, to differentiate between regular and irregular (missing "doneChargingTime" value)

In [151]:
charging_data_regular_trips = charging_data[charging_data['doneChargingTime'].notnull()]
charging_data_wo_doneChargingTime = charging_data[charging_data['doneChargingTime'].isnull()]

print(f"Regular trips: {charging_data_regular_trips.shape[0]}")
print(f"Irregular trips without doneChargingTime value: {charging_data_wo_doneChargingTime.shape[0]}")

Regular trips: 60950
Irregular trips without doneChargingTime value: 4087
